In [1]:
# Start with some imports - rich is a library for making formatted text output in the terminal

from rich.console import Console
from dotenv import load_dotenv
from openai import OpenAI
import json
load_dotenv(override=True)

True

In [3]:
def show(text):
    try:
        Console().print(text)
    except Exception:
        print(text)


In [6]:
openai = OpenAI()

In [7]:
todos, completed = [], []

In [15]:
def get_todo_report()->str:
    result = ""
    for index,todo in enumerate(todos):
        if completed[index]:
            result += f"Todo #{index+1}: [green][strike]{todo}[/strike][/green]\n"
        else:
            result += f"Todo #{index+1}: {todo}\n"
    show(result)
    return result

In [16]:
def create_todos(descriptions: list[str]) -> str:
    todos.extend(descriptions)
    completed.extend([False] * len(descriptions))
    return get_todo_report()

In [10]:
def mark_complete(index:int, completion_notes:str)-> str:
    if 1 <= index <= len(todos):
        completed[index-1]= True
    else:
        return "No to do at this Index"
    Console().print(completion_notes)
    return get_todo_report()    

In [17]:
create_todos(["Buy Grocery", "Complete Extra Lab", "Wrap up for the Day", "Spend time with Family"])

Todo #1: Buy Grocery
Todo #2: Complete Extra Lab
Todo #3: Wrap up for the Day
Todo #4: Spend time with Family
Todo #5: Buy Grocery
Todo #6: Complete Extra Lab
Todo #7: Wrap up for the Day
Todo #8: Spend time with Family
Todo #9: Buy Grocery
Todo #10: Complete Extra Lab
Todo #11: Wrap up for the Day
Todo #12: Spend time with Family

'Todo #1: [green][strike]Buy Grocery[/strike][/green]\nTodo #2: Complete Extra Lab\nTodo #3: Wrap up for the Day\nTodo #4: Spend time with Family\nTodo #5: Buy Grocery\nTodo #6: Complete Extra Lab\nTodo #7: Wrap up for the Day\nTodo #8: Spend time with Family\nTodo #9: Buy Grocery\nTodo #10: Complete Extra Lab\nTodo #11: Wrap up for the Day\nTodo #12: Spend time with Family\n'

In [18]:
mark_complete(3, "Wrapped Up")

Wrapped Up

Todo #1: Buy Grocery
Todo #2: Complete Extra Lab
Todo #3: Wrap up for the Day
Todo #4: Spend time with Family
Todo #5: Buy Grocery
Todo #6: Complete Extra Lab
Todo #7: Wrap up for the Day
Todo #8: Spend time with Family
Todo #9: Buy Grocery
Todo #10: Complete Extra Lab
Todo #11: Wrap up for the Day
Todo #12: Spend time with Family

'Todo #1: [green][strike]Buy Grocery[/strike][/green]\nTodo #2: Complete Extra Lab\nTodo #3: [green][strike]Wrap up for the Day[/strike][/green]\nTodo #4: Spend time with Family\nTodo #5: Buy Grocery\nTodo #6: Complete Extra Lab\nTodo #7: Wrap up for the Day\nTodo #8: Spend time with Family\nTodo #9: Buy Grocery\nTodo #10: Complete Extra Lab\nTodo #11: Wrap up for the Day\nTodo #12: Spend time with Family\n'

In [19]:
create_todos_json = {
    "name": "create_todos",
    "description": "Add new todos from a list of descriptions and return the full list",
    "parameters": {
        "type": "object",
        "properties": {
            "descriptions": {
                'type': 'array',
                'items': {'type': 'string'},
                'title': 'Descriptions'
                }
            },
        "required": ["descriptions"],
        "additionalProperties": False
    }
}

In [20]:
mark_complete_json = {
    "name": "mark_complete",
    "description": "Mark complete the todo at the given position (starting from 1) and return the full list",
    "parameters": {
        'properties': {
            'index': {
                'description': 'The 1-based index of the todo to mark as complete',
                'title': 'Index',
                'type': 'integer'
                },
            'completion_notes': {
                'description': 'Notes about how you completed the todo in rich console markup',
                'title': 'Completion Notes',
                'type': 'string'
                }
            },
        'required': ['index', 'completion_notes'],
        'type': 'object',
        'additionalProperties': False
    }
}

In [21]:
tools= [{"type":"function", "function":create_todos_json},
        {"type":"function","function":mark_complete_json}]

In [35]:
def handle_tool_calls(tool_calls):
    results = []
    for tool_call in tool_calls:
        tool_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        tool = globals().get(tool_name)
        result = tool(**arguments) if tool else {}
        results.append({"role":"tool", "content":json.dumps(result),"tool_call_id":tool_call.id})
    return results

In [37]:
def loop(messages):
    done = False
    while not done:
        response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages, tools=tools)
        finish_reason = response.choices[0].finish_reason
        if finish_reason=="tool_calls":
            message = response.choices[0].message
            tool_calls = message.tool_calls
            results = handle_tool_calls(tool_calls)
            messages.append(message)
            messages.extend(results)
        else:
            done = True
    show(response.choices[0].message.content)

In [44]:
system_message = """
You are given a problem to solve, by using your todo tools to plan a list of steps, then carrying out each step in turn.
Now use the todo list tools, create a plan, carry out the steps, and reply with the solution.
If any quantity isn't provided in the question, then include a step to come up with a reasonable estimate.
Provide your solution in Rich console markup without code blocks.
Do not ask the user questions or clarification; respond only with the answer after using your tools.
"""
user_message = """"
A bag contains 2 red, 3 green and 2 blue balls. Two balls are drawn at random. What is the probability that none of the balls drawn is blue
"""
messages = [{"role": "system", "content": system_message}, {"role": "user", "content": user_message}]

In [45]:
todos, completed = [], []
loop(messages)

Todo #1: Determine the total number of balls in the bag
Todo #2: Calculate the total number of ways to draw 2 balls from the bag
Todo #3: Calculate the number of ways to draw 2 balls that are not blue
Todo #4: Determine the probability that none of the balls drawn is blue

The total number of balls in the bag is 2 red + 3 green + 2 blue = 7 balls.

Todo #1: Determine the total number of balls in the bag
Todo #2: Calculate the total number of ways to draw 2 balls from the bag
Todo #3: Calculate the number of ways to draw 2 balls that are not blue
Todo #4: Determine the probability that none of the balls drawn is blue

The total number of ways to draw 2 balls from 7 is calculated using combinations: C(7, 2) = 21.

Todo #1: Determine the total number of balls in the bag
Todo #2: Calculate the total number of ways to draw 2 balls from the bag
Todo #3: Calculate the number of ways to draw 2 balls that are not blue
Todo #4: Determine the probability that none of the balls drawn is blue

The number of ways to draw 2 balls that are not blue: There are 2 red + 3 green = 5 non-blue balls. C(5, 2) = 10 
ways.

Todo #1: Determine the total number of balls in the bag
Todo #2: Calculate the total number of ways to draw 2 balls from the bag
Todo #3: Calculate the number of ways to draw 2 balls that are not blue
Todo #4: Determine the probability that none of the balls drawn is blue

The probability that none of the balls drawn is blue is calculated as: P = (Number of favorable outcomes) / (Total 
outcomes) = 10 / 21.

Todo #1: Determine the total number of balls in the bag
Todo #2: Calculate the total number of ways to draw 2 balls from the bag
Todo #3: Calculate the number of ways to draw 2 balls that are not blue
Todo #4: Determine the probability that none of the balls drawn is blue

The solution to the problem is as follows:

- **Total number of balls in the bag:** 7 (2 red + 3 green + 2 blue)
- **Total ways to draw 2 balls from the bag:** 21
- **Ways to draw 2 balls that are not blue:** 10 (from 5 non-blue balls)
- **Probability that none of the balls drawn is blue:** 10/21

Thus, the probability that none of the balls drawn is blue is **10/21**.